<h2>Optimization in Python: Pyomo tutorial<h2>

Q1: What is Pyomo?

- **Pyomo**: Python optimizaiton Modeling Objects
- framework for defining optimization models, not a solver!
- actively developed at Sandia National Laboratories
- part of the COIN-OR project

Q2: What are Pros & Cons of using Pyomo?

**Pros**: 
- (1) free as in beer.
- (2) cross platform which can be developed under Windows, Linux, and Mac
- (3) Supports many open source and commercial solvers
- (4) Supports declarative and imperative programming paradigm

**Cons**: 
- No built-in support for matrix/vector operation
- clunky handling of "jagged" sets 
- scalability issues
- limited support for fine-grain, solver-specific features
- more programming centric than some other modeling languages

In Pyomo, we also need to setup paramets, sets, variable constraints objective.


In [54]:
import pyomo.environ as pyo
import pyomo.environ as pe 
import pyomo.opt as po
import numpy as np
import gurobipy as grb
import pulp



In [55]:
solver = po.SolverFactory('cplex') # GNU linear programming kit



# Create a model for the following binary knapsack instance


\begin{align}
    & \max_{x_1, x_2, \dots, x_5} 3x_1 + 4x_2 + 5x_3 + 8x_4 + 9x_5\\
    & s.t ~~ 2x_1 + 3x_2 + 4x_3 + 7x_4 + 9x_5 \leq 20 \\
    & x_1, \dots, x_5 \in \{0,1\}
\end{align}


In [56]:

# specify the model
model = pe.ConcreteModel()
# specify the decision variables
model.x1 = pe.Var(domain=pe.Binary)
model.x2 = pe.Var(domain=pe.Binary)
model.x3 = pe.Var(domain=pe.Binary)
model.x4 = pe.Var(domain=pe.Binary)
model.x5 = pe.Var(domain=pe.Binary)

# specify the objective
obj = 3*model.x1 + 4*model.x2 + 5*model.x3 + 8*model.x4 + 9*model.x5
# sense of the optimization => maximize
# expr = expression of objective function
model.obj = pe.Objective(sense=pe.maximize, expr = obj)
constr = 2 * model.x1 + 3 * model.x2 + 4 * model.x3 + 7 * model.x4 + 9 * model.x5 <= 20
model.con = pe.Constraint(expr=constr)

results = solver.solve(model, tee=True)
print(pe.value(model.x1))
print(pe.value(model.x2))
print(pe.value(model.x3))
print(pe.value(model.x4))
print(pe.value(model.x5))
print(pe.value(model.obj))


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer Community Edition 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\jiami\AppData\Local\Temp\tmpkksmdbxm.cplex.log' open.
CPLEX> Problem 'C:\Users\jiami\AppData\Local\Temp\tmphvdyque9.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Problem name         : C:\Users\jiami\AppData\Local\Temp\tmphvdyque9.pyomo.lp
Objective sense      : Maximize
Variables            :       6  [Nneg: 1,  Binary: 5]
Objective nonzeros   :       5
Linear constraints   :       2  [Less: 1,  Equal: 1]
  Nonzeros           :       6
  RHS nonzeros       :       2

Variables            : Min LB: 0.000000         Max UB: 1.000000       
Objective nonzer

The best way to use Pyomo is to implement the general form of the problem. The instance above is generalized by the formulation below.
\begin{align}
    & \max_{x} \sum_{i} c_i x_i\\
    & s.t ~~ \sum_{i} a_i x_i \leq b \\
    & x_i\in \{0,1\}
\end{align}

In [70]:
model = pe.ConcreteModel()
model.N = pe.RangeSet(1,5)
c = np.random.rand(5)
a = np.random.rand(5)
c_dict = {i+1: value for i, value in enumerate(c)}
a_dict = {i+1: value for i, value in enumerate(a)}
b = 20

model.c = pe.Param(model.N, initialize = c_dict)
model.a = pe.Param(model.N, initialize = a_dict)
model.b = pe.Param(initialize = b)

# print(model.c)
# print(model.a)
# print(model.b.value)

model.x = pe.Var(model.N, domain=pe.Binary)
obj_expr = sum(model.c[i] * model.x[i] for i in model.N)
model.obj = pe.Objective(expr = obj_expr, sense=pe.maximize)

constr_expr = sum(model.a[i] * model.x[i] for i in model.N) <= model.b
model.constr = pe.Constraint(expr = constr_expr)

results = solver.solve(model)


for i in model.N:
    print(pe.value(model.x[i])) 
print(pe.value(model.obj))


1.0
1.0
1.0
1.0
1.0
3.0885892287440013
